# Assigment 3

In [1]:
import igl
import numpy as np
import meshplot as mp

In [2]:
v, f = igl.read_triangle_mesh("data/bunny.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

# Vertex normal

In [3]:
#Standard face normal
n = np.zeros_like(v)

# by default computes as uniform, specified just for clarity
n = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_UNIFORM)

# compute normals for each face
# accumulate the normal to the entries of n corresponding to the face's vertices
# normalize all entries
# save for later use

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [4]:
#Area-weighted face normal
n = np.zeros_like(v)

n = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)

#same as above, but faces' normals shall not be normalized prior to accumulate (cross product includes area)

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [18]:
# TBD: find right threshold

#Compute Laplacian matrix
#Compute normal direction from mean curvature formula
#If the norm if smaller then eps, substitute with the saved standard normal
#Normalize
#Adjust directions by using dot product with standard normal


def compute_mean_curvature_normals(v, f, eps=1e-6):
    # Compute cotangent-weighted Laplacian matrix
    L = igl.cotmatrix(v, f)
    # Compute mass matrix
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    # Compute mean curvature normals HN = inv(M) * (L.dot(v)) 
    # [https://libigl.github.io/tutorial/#curvature-directions]
    HN = -np.linalg.solve(M.toarray(), L.dot(v))  # shape (n,3)
    
    # Prepare fallback area-weighted normals
    fallback = igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)
    # Normalize and threshold
    N = np.zeros_like(HN)
    lengths = np.linalg.norm(HN, axis=1)
    for i in range(HN.shape[0]):
        if lengths[i] > eps:
            N[i] = HN[i] / lengths[i]
        else:
            N[i] = fallback[i]
        
    # fix signs
    sign_flip = np.sign((N * fallback).sum(axis=1))[:, None]
    return sign_flip * N

In [21]:
#Mean-curvature normal
n = np.zeros_like(v)

n = compute_mean_curvature_normals(v, f)
mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [24]:
from collections import deque

# This class is used to create an adjacency list for the vertices of a mesh
# we use it to find neighbours for PCA and quadratic fitting
class VertexAdjacencyList:
    def __init__(self, V, F):
        self.V = V
        self.F = F
        self.adjacency = [[] for _ in range(len(V))]
        self._build_adjacency()

    def _build_adjacency(self):
        for face in self.F:
            for i in range(3):
                v0, v1 = face[i], face[(i + 1) % 3]
                self.adjacency[v0].append(v1)
                self.adjacency[v1].append(v0)

    # Akin to k-nearest neighbors, but in the context of the mesh
    def k_nn(self, start_idx, k):
        visited = set([start_idx])
        queue = deque([start_idx])
        neighbors = []
        
        while queue and len(neighbors) < k:
            current = queue.popleft()
            for nbr in self.adjacency[current]:
                if nbr not in visited:
                    visited.add(nbr)
                    neighbors.append(nbr)
                    queue.append(nbr)
                    if len(neighbors) >= k:
                        break
        return neighbors
    
    # Akin to a k-ring, but in the context of the mesh
    def k_ring(self, start_idx, k):
        if k < 1:
            return set()

        visited = {start_idx}
        queue = deque([(start_idx, 0)])
        result = set()

        while queue:
            current, depth = queue.popleft()
            # Stop exploring deeper once we hit ring k
            if depth == k:
                continue

            for nbr in self.adjacency[current]:
                if nbr not in visited:
                    visited.add(nbr)
                    result.add(nbr)
                    queue.append((nbr, depth + 1))

        return list(result)

In [25]:
# TBD: analyze and refactor if needed, seems fine right now

def compute_pca_normals(v, f, k=16, adj=None):
    if adj is None:
        adj = VertexAdjacencyList(v, f)

    N = np.zeros_like(v)
    # for each vertex
    for i, vi in enumerate(v):

        # get neighbours
        neighbors = adj.k_ring(i, k)
        if len(neighbors) < 3:
            N[i] = np.array([0.0, 0.0, 1.0])
            continue
        pts = v[neighbors]

        # center the points
        pts -= pts.mean(axis=0)

        # compute PCA
        cov = np.cov(pts.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        
        # smallest eighenvector is the normal
        normal = eigvecs[:, np.argmin(eigvals)]
        N[i] = normal

    # fix signs
    stdN = igl.per_vertex_normals(v, f)
    sign_flip = np.sign((N * stdN).sum(axis=1))[:, None]
    return sign_flip * N

In [27]:
#PCA normal

n = np.zeros_like(v)
k = 3

n = compute_pca_normals(v, f, k)
#Define function to find kNN of a vertex (either by breadth-first expansion, or by exhaustive search)

#For each vertex:
#   find its kNN's
#   build the covariance matrix
#   find its eigenvectors by eigen decomposition
#   set normal from eigenvector corresponding to minimum eigenvalue
#Adjust directions as above

mp.plot(v, f, n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [28]:
# TBD: analyze and refactor if needed, seems fine right now

def compute_quadratic_normals(V, F, k=16, adj=None):
    if adj is None:
        adj = VertexAdjacencyList(V, F)

    N = np.zeros_like(V)
    stdN = igl.per_vertex_normals(V, F)

    # for each vertex
    for i, vi in enumerate(V):
        
        # get k neighbors
        neighbors = adj.k_ring(i, k)
        if len(neighbors) < 3:
            N[i] = np.array([0.0, 0.0, 1.0])
            continue

        pts = V[neighbors]
        
        # center points to mean of the neighbourhood
        pts -= pts.mean(axis=0)

        # compute PCA
        cov = np.cov(pts.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        e1, e2 = eigvecs[:, 1], eigvecs[:, 2]
        e3 = eigvecs[:, 0] # smallest eigenvector is the height

        # compute coordinates (u, v, h) in the local frame
        coords = np.dot(pts, np.vstack([e1, e2, e3]).T)
        u, v, h = coords[:,0], coords[:,1], coords[:,2]

        # we'll use as quadratic function h = a*u^2 + b*u*v + c*v^2 + d*u + e*v + f
        A = np.column_stack([u*u, u*v, v*v, u, v, np.ones_like(u)])
        # we want to fit the quadratic function to the height h
        coeffs, *_ = np.linalg.lstsq(A, h, rcond=None)
        # coeffs = [a, b, c, d, e, f]
        # we only care about d and e, because they are 
        # the derivatives in (0, 0), the vertex
        _, _, _, fu, fv, _ = coeffs

        # compute the tangent plane basis
        tu = e1 + fu * e3
        tv = e2 + fv * e3
        # the normal is the cross product of the tangent basis
        N[i] = np.cross(tu, tv)
        N[i] /= np.linalg.norm(N[i])

    # fix signs
    sign_flip = np.sign((N * stdN).sum(axis=1))[:, None]
    return sign_flip * N

In [30]:
#Quadratic fitting normal

n = np.zeros_like(v)
k = 3


n = compute_quadratic_normals(v, f, k)

#For each vertex:
#   find kNN's and compute PCA as before
#   express all neighbors in a local ref framewith origin at the vertex and axes aligned to the PCA
#   fit a quadratic polynomial to such data
#   compute the components of the Jacobian at the origin and express them in the global coordinate system
#   compute the normal as orthogonal to the Jacobian's components
#Adjust directions as above

mp.plot(v, f, n=n, shading={"flat": False})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

# Curvature

In [12]:
# TBD: try different color maps
# CHECK HOW PRINCIPAL CURVATURES ARE COMPUTED

#Obtain Gaussian curvature k from related igl function 
k = igl.gaussian_curvature(v, f)
#Compute mean curvature h from Laplacian as above
L = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
hn = -np.linalg.solve(m.toarray(), L.dot(v))
h = np.linalg.norm(hn, axis=1)
#Compute principal curvatures k1 and k2 from k and h
k1 = h - np.sqrt(h*h - k)
k2 = h + np.sqrt(h*h - k)

mp.plot(v, f, k, shading={"flat": False})
mp.plot(v, f, h, shading={"flat": False})
mp.plot(v, f, k1, shading={"flat": False})
mp.plot(v, f, k2, shading={"flat": False})
#Experiment with different color maps

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [13]:
#Obtain principal curvatures k1 and k2 and their directions v1 and v2 from related igl function
v1, v2, k1, k2 = igl.principal_curvature(v, f)
#Compute Gaussian curvature k and mean curvature h from k1 and k2
k = k1 * k2
h = (k1 + k2) / 2.0

# The following plots principal directions on top of a mesh colored with one of the curvatures
p = mp.plot(v, f, k1, shading={"wireframe": False}, return_plot=True)
avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})
#Experiment with colormaps for k2, k and h, too

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

2

# Smoothing with the Laplacian

In [14]:
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

In [15]:
# TBD: use uniform laplacian

# Explicit laplacian
ll = 0.0000001  # speed factor
it = 1000       # iterations

# MAIN FORMULA
# p_t+1 = (I + ll * m^-1 x L )p_t

# Compute stiffness matrix (cotmatrix) and mass matrix
L = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# Build matrix for forward Euler

# Invert the diagonal of the mass matrix
# Minv = m^-1
Minv = sp.diags(1 / m.diagonal())

# Build the operator 
# M^-1 * L
Euler_op = Minv @ L

# Run forward Euler on a copy vi of v
vi = v.copy()

# Run forward Euler iterations
for _ in range(it):
    vi += ll * (Euler_op @ vi)

# Plot the result
p = mp.plot(vi, f)

# Experiment with different speeds and iterations

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.018144…

In [16]:
# Implicit laplacian

ll = 0.001  # speed factor

# MAIN FORMULA
# (m - ll * L ) p_t+1 = m * p_t 

# Compute stiffness matrix (cotmatrix) and mass matrix
L = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# Build matrix and right hand side for backward Euler

# Left hand side
# A = M - ll * L
A = m - ll * L

# Right-hand side: 
# B = M * v
B = m @ v

# Solve the linear system 
# A * vi = B
vi = spsolve(A, B)

# Plot the result
p = mp.plot(vi, f)

# Experiment with different parameters

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.020321…